Дан набор звуковых файлов в WAV формате. Каждый звуковой файл продолжительностью одна секунда где произносится ровно одно из 10 английских слов. Набор слов: ['cat', 'dog', 'five', 'four', 'house', 'one', 'six', 'stop', 'three', 'two'].

Задача написать программу, которая автоматически определит какое именно слово произносится в звуковом файле.

В папке train лежат тренировочные файлы, разбитые по подпапкам. Именование папки совпадает со словом, которое произносится в звуковом файле. Название файлов имеет следующий формат словоIDчеловеканомер.

Замечание: люди, которые произносили слова в train и люди которые произносили слова в test различаются.

В файле sample_submission.csv приведен пример файла для отправки на сервер. Он содержит два поля id и answer. Необходимо вывести распознанное слово в поле answer (например 'cat').

- скачиваем библиотеку speechrecognition 
- документация: https://pypi.org/project/SpeechRecognition/
- страница на русском языке https://learn4kid-python.firebaseapp.com/python_data_structure/python_speech_recognition/#распознавание-речи-на-языках-отличных-от-англииского

In [ ]:
!pip install SpeechRecognition

импорт библиотек

In [ ]:
import speech_recognition as speech_r
import wave

import pandas as pd
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split

добавим список слов, которые произносятся в файлах

In [ ]:
array = ['cat', 'dog', 'five', 'four', 'house', 
       'one', 'six', 'stop', 'three', 'two']

получаем звуковые файлы сразу из теста

напишем функцию, которая получает на вход путь файла (относительный) и выдает распознанное слово
- audio_number - парсинг названия файла
- sample - объект аудиофайла для распознавания
- recognizer - экземпляр распознавателя речи
- result - слово-ответ
- audiodata - переформатируем/запишем аудиофайл sample в аудиодату для работы с методами распознавания
- text - распознаем какой-либо текст из аудиодаты и высвечиваем все возможные результаты
при непустом значении text смотрим, есть ли совпадения результата с исходным списком слов array. да - записываем в переменную результата, нет - записываем пустое значение NaN

In [ ]:
def recognize_from_path(path):
    audio_number = path.split('/')[1]
    sample = speech_r.WavFile(path)
    
    recognizer = speech_r.Recognizer()
    result = ""
    # Create audio data
    with sample as source:
        audiodata = recognizer.record(source)
        text = recognizer.recognize_google(audiodata, show_all = True)
    
    if text:
        for dictionary in text['alternative']:
            for transcript,val in dictionary.items():
                if transcript=='transcript' and val in array:
                    result = val
    else:
        result = "NaN"
                    
    return audio_number,result
    

## здесь работаем с очень (п**) огромной папкой со звуками которая добьет мой ноут окончательно.
- в файлес записываем список всех найденных в test папке файлов формата wav
- в n расчитываем количество зввуковых файлов для фора
- id, answers - для датафрейма-ответа
- потом проходимся по каждому пути файла(test+номер файла по порядку+.wav) и запускаем написанную функцию. в массивы id и answers записываем айди звука/файла и слово которое распознали.
- оформляем ответ в датафрейм
### крч заебалась ждать обработку 9к!!!! файлов так что если будешь доделывать то переделай строку с фором в for audio_num in range(n):
### все

In [ ]:
import glob

files = glob.glob("test/*.wav")
n = len(files)

ID, Answers = [],[]

for audio_num in range(n-9060):
    recognized_audio = recognize_from_path("test/"+str(audio_num)+".wav")
    name = recognized_audio[0]
    answer = recognized_audio[1]
    ID.append(name)
    Answers.append(answer)

res_df = pd.DataFrame({'id':ID, 'answer':Answers})
res_df

,id,answer
0,0.wav,NaN
1,1.wav,one
2,2.wav,two
3,3.wav,dog
4,4.wav,six
5,5.wav,stop
6,6.wav,stop
7,7.wav,NaN
8,8.wav,one
9,9.wav,six


сохраняем в файл submission_res

In [ ]:
res_df.to_csv('submission_res.csv', index=False)
res_df

,id,answer
0,0.wav,NaN
1,1.wav,one
2,2.wav,two
3,3.wav,dog
4,4.wav,six
5,5.wav,stop
6,6.wav,stop
7,7.wav,NaN
8,8.wav,one
9,9.wav,six
